In [1]:
import os
import numpy as np
import face_recognition as fr
import cv2
import csv
from datetime import datetime

In [2]:
path='attendance_students'
student_list=os.listdir(path)
student_list

['Aflah.jpg',
 'Akash Francis.jpeg',
 'Amala Raju.jpg',
 'Anakha cs.jpg',
 'Aromal pv.jpg',
 'Fizha.jpg',
 'Krishna Priya.jpeg',
 'Salman Faris.jpg',
 'Sinan Shereef.jpg',
 'Vishak s.jpg']

In [3]:
imgs=[]
student_names=[]
for i in student_list:
    img_path=os.path.join(path,i)
    img_read=cv2.imread(img_path)
    imgs.append(img_read)
    student_names.append(i.split('.')[0])

In [4]:
def face_encode(images):
    encode_list=[]
    for i in imgs:
        img1=cv2.cvtColor(i,cv2.COLOR_BGR2RGB)
        face_loc=fr.face_locations(img1)
        face_enc=fr.face_encodings(img1,face_loc)[0]
        encode_list.append(face_enc)
    return encode_list

In [5]:
known_face=face_encode(imgs)

In [6]:
marked_students=set()
attendance_file="attendance_2.csv"

In [7]:
def attendance_list(name):
    if name not in marked_students:
        now=datetime.now()
        time=now.strftime("%H:%M:%S")
        with open(attendance_file,'a',newline='') as f:
            writer=csv.writer(f)
            writer.writerow([name,time])
    marked_students.add(name)

In [8]:
video=cv2.VideoCapture(0)
while True:
    sucess,img=video.read()
    img2=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    face_locc=fr.face_locations(img2)
    face_encc=fr.face_encodings(img2,face_locc)
    
    for enc,loc in zip(face_encc,face_locc):
        matches=fr.compare_faces(known_face,enc)
        face_dis=fr.face_distance(known_face,enc)
        match_index=np.argmin(face_dis)
        names='unknown'
        if matches[match_index]==True:
            names=student_names[match_index]
            attendance_list(names)
            
        y1,x2,y2,x1=loc
        cv2.rectangle(img,(x1,y1),(x2,y2),(0,0,255),1)
        cv2.putText(img,names,(x1,y1),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1,(0,0,255),1)
    cv2.imshow('Frame',img)
    if cv2.waitKey(1)&0XFF==ord('q'):
        break
video.release()
cv2.destroyAllWindows()
    